# **#YouTubeScrap: A comprehensive tool for scraping YouTube data and transcript**

✅ This code, developed by [Ergon Cugler de Moraes Silva](https://github.com/ergoncugler) and [Isabela Rocha](https://www.linkedin.com/in/rocha-isabela/) (Brazil), aims to facilitate the `scraping, analysis, and organization of YouTube video data`. It is designed to extract information from video links, search queries, and metadata directly into Google Sheets. Key features include extracting video metrics (e.g., `title, views, likes, duration, channel details`), processing `transcripts` (when available), and allowing search queries filtered by date ranges. This tool is particularly useful for researchers, analysts, and content creators looking to gather and analyze data from YouTube efficiently and systematically.

✅ **The primary functionalities of the code include:** **1. Automated Worksheet Setup:** Automatically creates or loads a Google Sheet with predefined column headers, ensuring all extracted data is structured for analysis / **2. Search YouTube Videos:** Performs video searches based on user-defined queries, filtering results by date ranges and limits, and saves video links directly to the worksheet / **3. Extract Video Metrics:** Processes individual video links to gather detailed metadata, including channel name, number of subscribers, view count, like count, video tags, upload date, and more / **4. Extract Transcripts:** Automatically retrieves video transcripts (in multiple languages) and adds them to the worksheet for content analysis, when available.

✅ All extracted data is stored directly in a Google Sheet, making it accessible, shareable, and ready for further processing. The integration of tools like yt_dlp and YouTubeTranscriptAPI ensures accuracy, flexibility, and compliance with modern data formats. Researchers and analysts can benefit from its scalability and efficient handling of large datasets, avoiding the limitations of manual extraction.

✅ The code is open-source and freely available for use at [GitHub repository link (to be added)]. While the code can be freely modified and adapted, users are responsible for ensuring their usage complies with YouTube's terms of service and relevant data privacy regulations.

## **#Applications and Use Cases**
✅ The YouTubeScrap has a wide range of applications for research and analysis: **1. Academic Research:** Useful for exploring public discourse, analyzing political content, studying misinformation, or tracking media trends on YouTube / **2. Media Analysis:** Enables the extraction of video trends, engagement metrics, and channel growth for journalists and media professionals / **3. Social Science and Communication Studies:** Facilitates the study of digital communities, propaganda, and content production on YouTube platforms / **4. Content Creators and Marketers:** Provides insights into competitor performance, video optimization, and viewer engagement for strategic planning.

✅ To credit this academic work and the scraping code, it is recommended to cite: **SILVA, Ergon Cugler de Moraes; ROCHA, Isabela. *TelegramScrap: A comprehensive tool for scraping Telegram data*. (Dez) 2024. Available at: [https://github.com/ergoncugler/web-scraping-youtube/](https://github.com/ergoncugler/web-scraping-youtube/).**

In [ ]:
# @title **1. [ Required ] Create or load a Google Sheet** { display-mode: "form" }

# @markdown **This cell is required and must be run first** to initialize the Google Sheet used for storing data. It will either:
# @markdown 1. Create a new Google Sheet `sheet_name` with predefined columns, or
# @markdown 2. Load an existing sheet named in the `sheet_name` variable.

# @markdown You will need to **authorize access to Google Drive** for this to work. This is secure, and no data will be exposed or shared beyond this script. Once run, a link to your worksheet will be provided. Without this step, subsequent cells will not work correctly, as you will not have the spreadsheet available to allocate metrics and transcripts.


!pip install -q yt-dlp youtube-transcript-api gspread tqdm scrape-youtube

sheet_name = "brazilian_example" # @param {type:"string"}
from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

COLUMN_HEADERS = [
    "Link", "ID", "Title", "Full Title", "Description", "Channel Name", "Channel ID",
    "Channel URL", "Timestamp", "Publish Date", "Channel Name (Alt)", "Channel ID (Alt)",
    "Channel URL (Alt)", "Subscribers", "Is Verified", "Location", "Length (s)",
    "Views", "Likes", "Dislikes", "Reposts", "Comments", "Tags", "Thumbnail", "Transcript"
]

def load_or_create_sheet(sheet_name, headers):
    try:
        sheet = gc.open(sheet_name)
    except gspread.SpreadsheetNotFound:
        sheet = gc.create(sheet_name)
        print(f"Worksheet '{sheet_name}' created successfully.")
    worksheet = sheet.get_worksheet(0) or sheet.add_worksheet(title="Sheet1", rows="100", cols="26")
    current_headers = worksheet.row_values(1)
    if current_headers != headers:
        worksheet.insert_row(headers, 1)
    print(f"Access your worksheet here: https://docs.google.com/spreadsheets/d/{sheet.id}/edit")
    return worksheet

worksheet = load_or_create_sheet(sheet_name, COLUMN_HEADERS)


In [ ]:
# @title **2. [ Optional ] Search YouTube videos and save links**  *(with the worksheet already created in step 1)*{ display-mode: "form" }

# @markdown **This step is optional if you prefer to add links manually in the sheet.** Use this cell to **search YouTube videos** based on up to 5 queries and save their links in the Google Sheet.
# @markdown - Define search terms in `query_1`, `query_2`, etc. Use operators like `-` to exclude terms (e.g., `'election -trump'`).
# @markdown - Set `max_results` to limit the number of videos fetched per query (default: 10).
# @markdown - Use `start_date` and `end_date` to restrict the search to a specific date range. Leave these blank to fetch all available videos.

query_1 = "'presidente lula' -bolsonaro -eleição" # @param {type:"string"}
query_2 = "election usa -biden -kamala -trump" # @param {type:"string"}
query_3 = "" # @param {type:"string"}
query_4 = "" # @param {type:"string"}
query_5 = "" # @param {type:"string"}
max_results = 10 # @param {type:"slider", min:1, max:1000, step:1}
start_date = '2020-01-01' # @param {type:"date"}
end_date = '2024-12-31' # @param {type:"date"}

import scrapetube
from yt_dlp import YoutubeDL
from datetime import datetime
from tqdm.notebook import tqdm

queries = [query_1, query_2, query_3, query_4, query_5]
queries = [q for q in queries if q.strip()]  # Remove empty queries

date_start = datetime.strptime(start_date, "%Y-%m-%d")
date_end = datetime.strptime(end_date, "%Y-%m-%d")

def get_video_details(video_id):
    url = f"https://www.youtube.com/watch?v={video_id}"
    ydl_opts = {"quiet": True}
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return {
            "title": info.get("title", "Unknown"),
            "upload_date": info.get("upload_date", "00000000"),
            "link": url,
        }

def video_matches_date(video, date_start, date_end):
    try:
        upload_date = datetime.strptime(video["upload_date"], "%Y%m%d")
        return date_start <= upload_date <= date_end
    except:
        return False

def search_youtube_to_sheet(worksheet, queries, max_results, date_start, date_end):
    next_row = len(worksheet.col_values(1)) + 1
    for query in tqdm(queries, desc="Searching YouTube"):
        videos = scrapetube.get_search(query)
        results = []
        for video in tqdm(videos, desc=f"Processing '{query}'", leave=False):
            if len(results) >= max_results:
                break
            video_id = video["videoId"]
            video_details = get_video_details(video_id)
            if video_matches_date(video_details, date_start, date_end):
                results.append(video_details)
                worksheet.update_cell(next_row, 1, video_details["link"])
                next_row += 1

search_youtube_to_sheet(worksheet, queries, max_results, date_start, date_end)


In [ ]:
# @title **3. [ Optional ] Extract metrics from videos** *(with the worksheet already created in Step 1 and links either inserted manually or added in Step 2)* { display-mode: "form" }

# @markdown Run this cell to **extract metadata** for video links stored in the Google Sheet. Data such as `title`, `views`, `likes`, `channel name`, and `upload date` will be automatically fetched and added to their respective columns. The script processes only links without existing metadata, ensuring efficient updates. Make sure the Google Sheet contains valid YouTube links in the `Link` column before running this cell.


from yt_dlp import YoutubeDL
import re
from tqdm.notebook import tqdm

def extract_video_id(link):
    """
    Extracts the video ID from a YouTube link.

    Args:
        link (str): Full video URL.

    Returns:
        str: Extracted video ID.
    """
    match = re.search(r"v=([^&]+)", link)
    return match.group(1) if match else "Unknown"

def extract_video_metrics(worksheet):
    """
    Extracts video metrics and updates the worksheet.

    Args:
        worksheet: Google Worksheet object.
    """
    ydl_opts = {
        'quiet': True,
        'no_warnings': True,
        'skip_download': True,
    }

    # Collect all links with missing metadata
    links = []
    i = 2
    while worksheet.cell(i, 1).value:
        if not worksheet.cell(i, 2).value:
            links.append((i, worksheet.cell(i, 1).value))
        i += 1

    # Iterate over links and process each video
    for index, link in tqdm(links, desc="Extracting metrics"):
        try:
            video_id = extract_video_id(link)  # Extract video ID from the link

            with YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(link, download=False)

            tags = info.get('tags', [])
            tags_str = ', '.join(tags) if isinstance(tags, list) else 'Unknown'

            # Update worksheet with extracted data
            worksheet.update_cell(index, 2, video_id)  # Video ID
            worksheet.update_cell(index, 3, info.get('title', 'Unknown'))
            worksheet.update_cell(index, 4, info.get('fulltitle', 'Unknown'))
            worksheet.update_cell(index, 5, info.get('description', '').replace('\n', ' '))
            worksheet.update_cell(index, 6, info.get('uploader', 'Unknown'))
            worksheet.update_cell(index, 7, info.get('uploader_id', 'Unknown'))
            worksheet.update_cell(index, 8, info.get('uploader_url', 'Unknown'))
            worksheet.update_cell(index, 9, info.get('timestamp', 'Unknown'))
            worksheet.update_cell(index, 10, info.get('upload_date', 'Unknown'))
            worksheet.update_cell(index, 11, info.get('channel', 'Unknown'))
            worksheet.update_cell(index, 12, info.get('channel_id', 'Unknown'))
            worksheet.update_cell(index, 13, info.get('channel_url', 'Unknown'))
            worksheet.update_cell(index, 14, info.get('channel_follower_count', 'Unknown'))
            worksheet.update_cell(index, 15, info.get('channel_is_verified', 'FALSE'))
            worksheet.update_cell(index, 16, info.get('location', 'Unknown'))
            worksheet.update_cell(index, 17, info.get('duration', 'Unknown'))
            worksheet.update_cell(index, 18, info.get('view_count', 'Unknown'))
            worksheet.update_cell(index, 19, info.get('like_count', 'Unknown'))
            worksheet.update_cell(index, 20, info.get('dislike_count', 'Unknown'))
            worksheet.update_cell(index, 21, info.get('repost_count', 'Unknown'))
            worksheet.update_cell(index, 22, info.get('comment_count', 'Unknown'))
            worksheet.update_cell(index, 23, tags_str)
            worksheet.update_cell(index, 24, info.get('thumbnail', 'Unknown'))
        except Exception as e:
            print(f"Error processing link {link}: {e}")

print("Processing metrics...")
extract_video_metrics(worksheet)
print("Metrics extracted successfully!")


In [ ]:
# @title **4. [ Optional ] Extract video transcripts** *(with the worksheet already created in Step 1 and links either inserted manually or added in Step 2)* { display-mode: "form" }

# @markdown Use this cell to retrieve **video transcripts** (if available) in supported languages like `English`, `Spanish`, and `Portuguese`. Transcripts will be added to the `Transcript` column in the Google Sheet. If no transcript is found, the entry will be marked as `[no subtitles]`. The script processes only rows without existing transcripts, skipping completed entries.

from youtube_transcript_api import YouTubeTranscriptApi
import re
from tqdm.notebook import tqdm

def extract_video_transcripts(worksheet):
    col_url = worksheet.col_values(1)
    urls_to_process = [(idx, url) for idx, url in enumerate(col_url[1:], start=2) if not worksheet.cell(idx, 25).value]

    for idx, url in tqdm(urls_to_process, desc="Extracting transcripts"):
        video_id = re.search(r"v=([^&]+)", url)
        video_id = video_id.group(1) if video_id else None
        if video_id:
            try:
                transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en', 'es', 'pt'])
                transcript_text = ' '.join([item['text'].replace('\n', ' ') for item in transcript])
                worksheet.update_cell(idx, 25, transcript_text)
            except Exception:
                worksheet.update_cell(idx, 25, "[no subtitles]")

extract_video_transcripts(worksheet)
